In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
pip install datasets


In [3]:
from datasets import load_dataset


dataset = load_dataset('casehold/casehold')
print(dataset)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for casehold/casehold contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/casehold/casehold.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['example_id', 'citing_prompt', 'holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4', 'label'],
        num_rows: 42509
    })
    validation: Dataset({
        features: ['example_id', 'citing_prompt', 'holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4', 'label'],
        num_rows: 5314
    })
    test: Dataset({
        features: ['example_id', 'citing_prompt', 'holding_0', 'holding_1', 'holding_2', 'holding_3', 'holding_4', 'label'],
        num_rows: 5314
    })
})


In [21]:
def convert_labels_to_int(example):
    example['label'] = int(example['label'])
    return example


dataset = dataset.map(convert_labels_to_int,batched=False)

Map:   0%|          | 0/42509 [00:00<?, ? examples/s]

Map:   0%|          | 0/5314 [00:00<?, ? examples/s]

Map:   0%|          | 0/5314 [00:00<?, ? examples/s]

In [22]:
small_train_dataset = dataset['train'].select(range(1000))
small_eval_dataset = dataset['validation'].select(range(500))

In [23]:
print(small_train_dataset[0])

{'example_id': 0, 'citing_prompt': "Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,", 'holding_0': 'holding that possession of a pipe bomb is a crime of violence for purposes of 18 usc  3142f1', 'h

In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [26]:


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    print(f"Sample input: {examples['citing_prompt'][:2]}")


    tokenized = tokenizer(examples['citing_prompt'], padding="max_length", truncation=True, max_length=256)
    print(f"Tokenized output: {tokenized.keys()}")

    return tokenized



tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Sample input: ["Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,", 'Colameta used customer information that he took from Protégé. Additionally, Colameta admits to having taken at least two Protégé p

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Sample input: ['was acting to obtain a benefit on behalf of a charitable ... organization.” U.S.S.G. § 2B1.1 cmt. 8(B). As the district court saw it and as the government sees it, Webster deserves the enhancement. He pretended to “act[ ] on behalf of a charitable ... organization,” U.S.S.G. § 2Bl.l(b)(9)(A), when he solicited personal information from the victims on behalf of fake charities. As Webster sees it, the enhancement does not apply. In his view, the commentary limits the application of the charity enhancement, and he was not acting to obtain a benefit on behalf of a charitable organization (as the commentary seems to require). As a general matter, the text of a guideline trumps commentary about it. See Stinson v. United States, 508 U.S. 36, 38, 113 S.Ct. 1913, 123 L.Ed.2d 598 (1993) (<HOLDING>). But we need not resolve whether the', 'a number of integers must be factored into the calculus— e.g., the nature of the information, the nature and characteristics of the supposed cri

In [27]:
def remove_labels(examples):
    return {
        'input_ids': examples['input_ids'],
        'attention_mask': examples['attention_mask']
    }

In [28]:
tokenized_small_train_dataset = tokenized_small_train_dataset.map(remove_labels, batched=True)
tokenized_small_eval_dataset = tokenized_small_eval_dataset.map(remove_labels, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [29]:
print(tokenized_small_train_dataset[0])


{'example_id': 0, 'citing_prompt': "Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,", 'holding_0': 'holding that possession of a pipe bomb is a crime of violence for purposes of 18 usc  3142f1', 'h

In [30]:

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()






In [31]:
def custom_data_collator(features):

  if 'labels' in features[0]:
    del features[0]['labels']
  return tokenizer.pad(
      features,
      padding="max_length",
      max_length=256,
      return_tensors="pt"
  )

In [33]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2ForSequenceClassification


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


config = GPT2Config.from_pretrained('gpt2')
config.pad_token_id = tokenizer.pad_token_id
config.num_labels = 5


model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

def preprocess_function(examples):
    output = tokenizer(
        examples['citing_prompt'],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )
    output['labels'] = [int(label) for label in examples['label']]
    return output
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=500,
)


tokenized_small_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
tokenized_small_eval_dataset = small_eval_dataset.map(preprocess_function, batched=True)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DefaultDataCollator(),
    train_dataset=tokenized_small_train_dataset,
    eval_dataset=tokenized_small_eval_dataset
)


trainer.train()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,1.646800
2,No log,1.702607


Epoch,Training Loss,Validation Loss
1,No log,1.646800
2,No log,1.702607
3,No log,1.734449


TrainOutput(global_step=375, training_loss=1.6596993815104166, metrics={'train_runtime': 9557.8448, 'train_samples_per_second': 0.314, 'train_steps_per_second': 0.039, 'total_flos': 391955742720000.0, 'train_loss': 1.6596993815104166, 'epoch': 3.0})

In [46]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


prompt = "In conclusion, the court finds that the defendant was guilty because"


inputs = tokenizer(prompt, return_tensors="pt")


outputs = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=150,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_p=0.85,
    no_repeat_ngram_size=5,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.eos_token_id
)


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


In conclusion, the court finds that the defendant was guilty because he had a prior conviction for an offense which resulted in his being placed on probation.
This case is not one of those cases where prosecutors believe there's any evidence to support their claims against you and your family or others who are accused under federal law; however it does make sense when looking at these facts as they relate directly to this particular situation - especially considering how much more extensive information we can gather from them than what I've seen before about other people charged with similar offenses by our government (and all too often prosecuted) during my time here within Michigan state courts:

I have heard many stories over the years concerning alleged rape victims going missing after getting into trouble...but never were reports
